# Update gridVeg Image Metadata in BigQuery

This notebook appends new image metadata records to the BigQuery table from a CSV file stored in GCS.

**Operation**: APPEND new rows (not replace entire table)

## Requirements
- Google Cloud credentials configured
- Configuration file: copy `config.example.yml` to `config.yml` and fill in your values
- Required packages: google-cloud-bigquery, google-cloud-storage, pandas, pyyaml


In [ ]:
# Import required libraries
import yaml
import pandas as pd
from pathlib import Path
from google.cloud import bigquery
from google.cloud import storage
from datetime import datetime

print("Libraries imported successfully")


## Load Configuration

**TODO**: Add configuration section to config.yml for this table


In [ ]:
# Load configuration from YAML file
config_path = Path("../config.yml")

if not config_path.exists():
    raise FileNotFoundError(
        f"Configuration file not found: {config_path}\n"
        "Please copy config.example.yml to config.yml and fill in your values."
    )

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Extract configuration values for gridVeg image metadata
# TODO: Update these config keys once added to config.yml
GCS_CSV_URL = config['gridveg_image_metadata']['gcs']['csv_url']
BACKUP_BUCKET = config['gridveg_image_metadata']['gcs'].get('backup_bucket')
BACKUP_PREFIX = config['gridveg_image_metadata']['gcs'].get('backup_prefix', 'backups/gridveg_image_metadata')
BQ_TABLE_ID = config['gridveg_image_metadata']['bigquery']['table_id']
BQ_PROJECT = config['gridveg_image_metadata']['bigquery'].get('project')

# Verify required config values
if not GCS_CSV_URL or GCS_CSV_URL.startswith('gs://your-'):
    raise ValueError("Please configure gridveg_image_metadata.gcs.csv_url in config.yml")
if not BQ_TABLE_ID or 'your-project' in BQ_TABLE_ID:
    raise ValueError("Please configure gridveg_image_metadata.bigquery.table_id in config.yml")

print("✓ Configuration loaded successfully")
print(f"  CSV URL: {GCS_CSV_URL[:60]}..." if len(GCS_CSV_URL) > 60 else f"  CSV URL: {GCS_CSV_URL}")
print(f"  Table ID: {BQ_TABLE_ID}")
print(f"  Backup: gs://{BACKUP_BUCKET}/{BACKUP_PREFIX}" if BACKUP_BUCKET else "  Backup: Not configured")


In [ ]:
# Initialize clients
bq_client = bigquery.Client(project=BQ_PROJECT) if BQ_PROJECT else bigquery.Client()
storage_client = storage.Client(project=BQ_PROJECT) if BQ_PROJECT else storage.Client()

print(f"✓ Clients initialized")
print(f"  Project: {bq_client.project}")


## Load CSV Data from GCS

Read the source CSV file containing new image metadata records.


In [ ]:
# Read CSV from GCS (new data)
print("Reading CSV from GCS...")
df_new = pd.read_csv(GCS_CSV_URL)

print(f"✓ CSV loaded successfully:")
print(f"  Rows: {len(df_new)}")
print(f"  Columns: {list(df_new.columns)}")
print(f"\nFirst few rows:")
df_new.head()


## Transform CSV Data

Apply schema transformations to match BigQuery table:
- Rename columns to match destination schema
- Convert date format from mm/dd/yy to ISO format (YYYY-MM-DD)
- Add image_url column (initially NULL - to be populated separately)
- Clean up Direction field (handle invisible character issue in "North")


In [ ]:
# Define column mapping from CSV to BigQuery
column_mapping = {
    '__kp_Photos': 'image_ID',
    'Survey Data::__kp_Survey': 'survey_ID',
    'Survey Data::SurveyDate': 'date',
    'Survey Data::SurveyYear': 'year',
    'Survey Data::_kf_Site': 'grid_point',
    'Direction': 'image_direction'
}

print("Column mapping:")
for csv_col, bq_col in column_mapping.items():
    print(f"  {csv_col:35s} → {bq_col}")


In [ ]:
# Verify CSV columns match expected schema
expected_csv_columns = set(column_mapping.keys())
actual_csv_columns = set(df_new.columns)

if actual_csv_columns == expected_csv_columns:
    print("✓ CSV columns match expected schema")
else:
    print("⚠ CSV column differences detected:")
    if actual_csv_columns - expected_csv_columns:
        print(f"  Unexpected columns: {actual_csv_columns - expected_csv_columns}")
    if expected_csv_columns - actual_csv_columns:
        print(f"  Missing columns: {expected_csv_columns - actual_csv_columns}")
    
print(f"\nCSV columns: {list(df_new.columns)}")


In [ ]:
# Apply transformation: rename columns
df_transformed = df_new.copy()
df_transformed = df_transformed.rename(columns=column_mapping)

print("✓ Columns renamed")
print(f"  Transformed columns: {list(df_transformed.columns)}")


In [ ]:
# Convert date from m/d/yy to proper datetime/date format
# Explicitly specify format to avoid parsing warnings and ensure consistency
# Note: %y handles 2-digit years (00-68 = 2000-2068, 69-99 = 1969-1999)
df_transformed['date'] = pd.to_datetime(df_transformed['date'], format='%m/%d/%y').dt.date

print("✓ Date format converted to date type")
print(f"  Sample dates: {df_transformed['date'].head().tolist()}")


In [ ]:
# Clean up Direction field - strip whitespace and handle invisible characters
# The source mentions "invisible difference in North" that displays as two levels
df_transformed['image_direction'] = df_transformed['image_direction'].str.strip()

# Check for unique values and any issues
print("✓ Direction field cleaned")
print(f"  Unique directions: {sorted(df_transformed['image_direction'].dropna().unique())}")
print(f"  Direction counts:")
for direction, count in df_transformed['image_direction'].value_counts().items():
    print(f"    {repr(direction):12s}: {count}")


In [ ]:
# Add image_url column (not in source CSV - will be NULL initially)
# This column exists in the destination schema but not in the source data
df_transformed['image_url'] = None

print("✓ Added image_url column (initially NULL)")
print(f"  Column will be populated separately with actual image URLs")


In [ ]:
# Reorder columns to match destination schema
expected_column_order = ['image_ID', 'image_url', 'survey_ID', 'date', 'year', 'grid_point', 'image_direction']
df_transformed = df_transformed[expected_column_order]

print("✓ Columns reordered to match destination schema")
print(f"  Final columns: {list(df_transformed.columns)}")


In [ ]:
# Display transformed data info
print("Transformed Data Info:")
df_transformed.info()
print(f"\nTransformed data preview:")
df_transformed.head()


## Read Existing BigQuery Table

Load the current data from BigQuery to compare with the new data.


In [ ]:
# Read existing data from BigQuery
print(f"Reading existing data from {BQ_TABLE_ID}...")
query = f"SELECT * FROM `{BQ_TABLE_ID}`"

try:
    df_existing = bq_client.query(query).to_dataframe()
    print(f"✓ Existing table loaded:")
    print(f"  Rows: {len(df_existing)}")
    print(f"  Columns: {list(df_existing.columns)}")
    print(f"\nExisting data preview:")
    display(df_existing.head())
except Exception as e:
    print(f"⚠ Error reading table: {e}")
    print("  This may be expected if the table doesn't exist yet.")
    df_existing = None


In [ ]:
# Display existing data info (if available)
if df_existing is not None:
    print("Existing Data Info:")
    df_existing.info()


## Compare New vs Existing Data

Identify which rows in the new data are not already in the existing table.


In [ ]:
# Compare datasets
if df_existing is not None:
    print("=== Comparison Summary ===\n")
    
    # Row count comparison
    print(f"Row count:")
    print(f"  Existing: {len(df_existing)}")
    print(f"  New CSV:  {len(df_transformed)}")
    
    # Column comparison
    existing_cols = set(df_existing.columns)
    new_cols = set(df_transformed.columns)
    
    if existing_cols == new_cols:
        print(f"\n✓ Columns match ({len(new_cols)} columns)")
    else:
        print("\n⚠ Column differences detected:")
        if new_cols - existing_cols:
            print(f"  New columns: {new_cols - existing_cols}")
        if existing_cols - new_cols:
            print(f"  Missing columns: {existing_cols - new_cols}")
    
    print(f"\nColumns: {list(df_transformed.columns)}")
else:
    print("No existing data to compare - this will be a new table creation.")


In [ ]:
# Identify new records (not in existing table)
# Use image_ID as the unique key (described as unique in source schema)
if df_existing is not None:
    existing_ids = set(df_existing['image_ID'])
    new_ids = set(df_transformed['image_ID'])
    
    # Find records in new data that aren't in existing
    ids_to_append = new_ids - existing_ids
    
    if ids_to_append:
        df_to_append = df_transformed[df_transformed['image_ID'].isin(ids_to_append)].copy()
        
        print(f"✓ Found {len(df_to_append)} new records to append")
        
        # Show year breakdown of new records
        print(f"\nNew records by year:")
        year_counts = df_to_append['year'].value_counts().sort_index()
        for year, count in year_counts.items():
            print(f"  {year}: {count} records")
        
        # Show direction breakdown
        print(f"\nNew records by direction:")
        direction_counts = df_to_append['image_direction'].value_counts()
        for direction, count in direction_counts.items():
            print(f"  {direction}: {count} records")
        
        print(f"\nSample of new records:")
        display(df_to_append.head(10))
    else:
        df_to_append = None
        print("⚠ No new records found - all records already exist in table")
        print("  Nothing to append.")
    
    # Check for any duplicates
    duplicate_ids = existing_ids & new_ids
    if duplicate_ids:
        print(f"\n⚠ Warning: {len(duplicate_ids)} records already exist in table")
        print(f"  These will be skipped during append.")
        if len(duplicate_ids) <= 10:
            print(f"\n  Sample duplicate image_IDs:")
            for img_id in list(duplicate_ids)[:10]:
                print(f"    {img_id}")
else:
    # No existing table, so all records are new
    df_to_append = df_transformed.copy()
    print(f"✓ No existing table - will create new table with {len(df_to_append)} records")
